# 1. Environment Setting

## (1). 라이브러리 설정

In [1]:
import re
import pandas as pd

from hanspell import spell_checker
from soynlp.normalizer import emoticon_normalize

## (2). 데이터 불러오기

In [2]:
train_dataset = pd.read_excel('/Users/hyeontae/Desktop/toy_ai/Dataset/감성대화말뭉치(최종데이터)_Training.xlsx', index_col = 0)
test_dataset = pd.read_excel('/Users/hyeontae/Desktop/toy_ai/Dataset/감성대화말뭉치(최종데이터)_Validation.xlsx', index_col = 0)

# 2. Data Preprocessing

## (1) 컬럼 추출

In [4]:
col = ['감정_대분류', '사람문장1']

train_data = train_dataset[col]
test_data = test_dataset[col]

## (2) 컬럼명 변경

In [5]:
train_data = train_data.rename(columns={'감정_대분류' : 'emotion', '사람문장1' : 'sentence'})
test_data = test_data.rename(columns={'감정_대분류' : 'emotion', '사람문장1' : 'sentence'})

## (3) 행 추출

- 필요한 라벨값을 가지는 행만 추출

In [6]:
need_label = ['분노', '기쁨', '불안', '슬픔']
train_data = train_data.loc[train_data['emotion'].isin(need_label)]
test_data = test_data.loc[test_data['emotion'].isin(need_label)]

- 인덱스 재정의

In [47]:
train_data = train_data.reset_index(drop = True)
test_data = test_data.reset_index(drop = True)

## (4) 텍스트 수정
- 맞춤법 수정
- 반복되는 문자 정제

- 텍스트 수정을 위한 함수 생성

In [49]:
def correct_spelling(sentence) :
    spelled_sent = spell_checker.check(sentence)
    hanspell_sent = spelled_sent.checked
    modified_sentence = emoticon_normalize(hanspell_sent, num_repeats = 2)
    return modified_sentence

- 텍스트 수정

In [50]:
train_data['sentence'] = train_data.sentence.apply(lambda sent : correct_spelling(sent))
test_data['sentence'] = test_data.sentence.apply(lambda sent : correct_spelling(sent))

## (5) 특수문자 제거

In [6]:
train_data['sentence'] = train_data.sentence.apply(lambda x : re.sub('[,.!?]', '', x))
test_data['sentence'] = test_data.sentence.apply(lambda x : re.sub('[,.!?]', '', x))

In [8]:
train_data.to_csv('/Users/hyeontae/Desktop/toy_ai/preprocessed_train_data.csv', index = False)
test_data.to_csv('/Users/hyeontae/Desktop/toy_ai/preprocessed_test_data.csv', index = False)